In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
tf.logging.set_verbosity(tf.logging.ERROR)

In [3]:
tf.enable_eager_execution()

In [ ]:
tfds.list_builders()

In [4]:
mnist_train, mnist_test = tfds.load('mnist', split=['train', 'test'])

In [5]:
def parser(example):
    return example["image"] / 255, example["label"]

In [6]:
mnist_train = mnist_train.shuffle(1000).map(parser).batch(10).prefetch(10)

In [10]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), input_shape=(28, 28, 1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, (3, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [11]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [12]:
mnist_iterator = mnist_train.make_one_shot_iterator()

In [13]:
model.fit(x=mnist_iterator, steps_per_epoch=6000, epochs=1)

1739/6000 [=======>......................] - ETA: 1:41 - loss: 0.2336 - acc: 0.9301

KeyboardInterrupt: 

In [ ]:
mnist_test = mnist_test.map(parser).batch(20).prefetch(10)

In [ ]:
eval_iterator = mnist_test.make_one_shot_iterator()

In [ ]:
model.evaluate(eval_iterator, steps=10000//20)

In [ ]:
new_iterator = mnist_train.make_one_shot_iterator()

In [ ]:
images, labels = next(new_iterator)

In [ ]:
predictions = model.predict(x=images, steps=1)

In [ ]:
labels

In [ ]:
for features in mnist_train:
    image, label = features["image"], features["label"]
    model.fit(x=image, y=label, steps_per_epoch=6000, epochs=1)